# Investment and Renewable Energy Support

This week we are looking at investment in electricity markets. Specifically, we will look at how we can understand and model investment incentives, and how policy can influence these. We will first look at a simple way of thinking about investment, using screening curves. We will then see an example of a more complex modelling method which can include much more detail.

## Screening curves

Electricity demand fluctuates from hour to hour. Because of this, an optimal electricity generation mix, i.e. the particular combination of power plants that can serve the market, will likely consist of a number of different types of power plants. Some will be used most of the time - these are base load power plants, which generally have high investment costs but low marginal cost. Then there are mid-load plants, which are used often but not all the time (with medium investment costs and medium marginal costs) and peak load power plants which are only used when demand is very high. Peak load power plants generally have a low investment cost, but a high marginal cost. An optimal (i.e. lowest total cost) generation mix generally has all these types, because it is not worth building expensive base load power plants to cover all demand (as some of them will not be used all the time), but it’s also a waste of money to only use peak load plants as they might be cheap to build, but if you’re running them all the time this is false economy, as their running costs are high.

Screening curves can visualise the trade-off between investment and operational costs and show what the optimal mix of technologies might look like. Before we look at one, we need a representation of energy demand. A time series of electricity demand might look like this (I have plotted 100 hours here for demonstration only - of course for real-world investment purposes you’d want data to cover several years, although in larger models it is common to cluster that data into a smaller number clusters consisting of similar hours to save computation time): of money to only use peak load plants as they might be cheap to build, but if you’re running them all the time this is false economy, as their running costs are high. Screening curves can visualise the trade-off between investment and operational costs and show what the optimal mix of technologies might look like. Before we look at one, we need a representation of energy demand. A time series of electricity demand might look like this (I have plotted 100 hours here for demonstration only - of course for real-world investment purposes you’d want data to cover several years, although in larger models it is common to cluster that data into a smaller number clusters consisting of similar hours to save computation time):

In [6]:
import pandas as pd
import datetime
import numpy as np
import hvplot.pandas
import plotly.graph_objects as go

# Sample data (replace this with your actual data)
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')  # Use hourly frequency
hours_in_day = 24

# Simulate higher demand in winter than summer and daily peaks
base_demand = [
    120 + 10 * np.sin((i % (365 * hours_in_day)) * 2 * np.pi / (365 * hours_in_day)) +
    30 * ((7 <= i % hours_in_day <= 11) or (17 <= i % hours_in_day <= 20))
    for i in range(len(date_range))
]

# Introduce random noise
np.random.seed(42)  # Set seed for reproducibility
noise = np.random.normal(loc=0, scale=5, size=len(date_range))  # Adjust the scale for the amount of noise
demand_with_noise = base_demand + noise

data = {'Datetime': date_range, 'Electrical Demand': demand_with_noise}

df = pd.DataFrame(data)

# Create a line chart using hvplot
hv_plot = df.hvplot.line(
    x='Datetime',
    y='Electrical Demand',
    title='Electrical Demand Over a Year with Seasonal and Daily Peaks',
    xlabel='Datetime',
    ylabel='Demand (Units)'
)

# Convert hvplot to Plotly figure
fig = hv_plot.to.plotly()

# Show the plot
fig

AttributeError: 'ElementConversion' object has no attribute 'plotly'

If we now sort this time series, arranging the 100 hours in order from largest to smallest demand levels, we obtain a load duration curve:

In [2]:
import plotly.express as px
import pandas as pd
import datetime
import numpy as np

# Sample data (replace this with your actual data)
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start=start_date, end=end_date, freq='H')  # Use hourly frequency
hours_in_day = 24

# Simulate higher demand in winter than summer and daily peaks
base_demand = [
    120 + 10 * np.sin((i % (365 * hours_in_day)) * 2 * np.pi / (365 * hours_in_day)) +
    30 * ((7 <= i % hours_in_day <= 11) or (17 <= i % hours_in_day <= 20))
    for i in range(len(date_range))
]

# Introduce random noise
np.random.seed(42)  # Set seed for reproducibility
noise = np.random.normal(loc=0, scale=5, size=len(date_range))  # Adjust the scale for the amount of noise
demand_with_noise = base_demand + noise

data = {'Datetime': date_range, 'Electrical Demand': demand_with_noise}

df = pd.DataFrame(data)

# Sort the data by demand in descending order
df = df.sort_values(by='Electrical Demand', ascending=False)

# Calculate the cumulative percentage of time
df['Cumulative Percentage'] = (df['Electrical Demand'].rank(ascending=False, method='max') / len(df)) * 100

# Create a load duration curve using Plotly Express
fig = px.line(df, x='Cumulative Percentage', y='Electrical Demand', title='Load Duration Curve',
              labels={'Electrical Demand': 'Demand (Units)', 'Cumulative Percentage': 'Cumulative Percentage'})

# Show the plot
fig

From this we can see that, for instance, demand exceeds 10GW in only around 5 of the 100 hours, so around 5% of the time. We can now combine this load duration curve with a screening curve, which is simply a plot of the total (investment + fuel) cost of different generation technologies as a function of the number of hours the technologies are used. Consider the example below. The bottom diagram in this example if the screening curve - the load duration curve is on top. The screening curve shows, for the default options in this case, that gas capacity is the cheapest technology if it is used less than about 30 hours, nuclear is cheaper if it is used more than about 45 hours, and coal is cheaper if it is used between 30-45 and hours. If we find the break-even points between the technologies and map them on the load duration curve, we can see that,

- because demand is above 4.9GW for 45 hours, an optimal solution includes 4.9GW of nuclear capacity. If we would install more, this capacity would be used for less than 45 hours, in which case, as we can see in the screening curve, it is better to choose another technology
- because demand is above 6.1GW for 30 hours, an optimal solution includes 6.1 - 4.9 = 1.2GW of coal-fired capacity.
- the remaining 8GW of generation capacity needed to cover peak demand will be satisfied with gas-fired capacity, as this capacity will be used less than 30 hours.

The widget will let you explore this further. You can impose a nuclear or gas capacity subsidy (which decreases the fixed costs of production - or if you set it to a negative level it function as a tax which increases costs) and/or generation subsidy (which decreases, or increases if negative, the marginal cost of generation). Can you find a set of subsidies/taxes that drives coal out of the market? Can you achieve that with only capacity or generation subsidies, or do you need both? Can you shift the whole system to use only one type of generation? Do you understand how the optimal capacities change in response to your policy choices?

In [3]:
import plotly.graph_objects as go

# Data for different thermal power plant technologies
full_load_hours = [i for i in range(0, 8001, 1000)]  # 0 to 8000 in increments of 1000

# Linear equation for total annual cost based on run hours
def calculate_costs(run_hours, intercept, slope):
    return [intercept + slope * hours for hours in run_hours]

# Parameters for each technology
natural_gas_intercept = 100
natural_gas_slope = 0.06

coal_intercept = 150
coal_slope = 0.04

nuclear_intercept = 300
nuclear_slope = 0.01

# Calculate total annual costs for each technology
natural_gas_costs = calculate_costs(full_load_hours, natural_gas_intercept, natural_gas_slope)
coal_costs = calculate_costs(full_load_hours, coal_intercept, coal_slope)
nuclear_costs = calculate_costs(full_load_hours, nuclear_intercept, nuclear_slope)

# Create traces for each technology
trace_gas = go.Scatter(x=full_load_hours, y=natural_gas_costs, mode='lines+markers', name='Natural Gas')
trace_coal = go.Scatter(x=full_load_hours, y=coal_costs, mode='lines+markers', name='Coal')
trace_nuclear = go.Scatter(x=full_load_hours, y=nuclear_costs, mode='lines+markers', name='Nuclear')

# Create layout
layout = go.Layout(
    title='Screening Curves for Thermal Power Plant Technologies',
    xaxis=dict(title='Full Load Hours'),
    yaxis=dict(title='Total Annual Costs'),
    legend=dict(x=0, y=1),
)

# Create figure
fig = go.Figure(data=[trace_gas, trace_coal, trace_nuclear], layout=layout)

# Show the figure
fig

The example above had no renewables, but these can easily be included as long as it is never optimal to curtail renewables. Renewable production can simply be subtracted from demand to generate a net demand curve, which will lead to a different optimal conventional generation mix. Existing generation capacity can also be included - this would have a zero investment cost (as it already exist).

## Investment stage in market models

Screening curves are a good first approximation to analyse optimal investment in electricity markets - and therefore also investment levels we might see in a perfectly competitive market. However, there are many things that cannot be included in screening curves. Because the demand data is sorted to obtain the load duration curve, all information about the temporal sequence of time period is lost. Technologies such as storage, which shift demand from one time period to the next, are therefore difficult to include, as are ramping constraints, start-ups, etc. Grid constraints, which can limit how power plants can be used, can also not be included. 

For a more detailed analysis of investment in electricity markets (and therefore, for instance, also the effect of policies that target investment), it is possible to extend the market models that we have seen before to include an investment stage. In the widget below, you can explore this. I have taken one of the optimal dispatch models from the previous weeks, but gave the model the option to not just dispatch existing plants, but also to invest in new capacity (which can then subsequently be dispatched - hence ‘multi-level’). For simplicity, I’ve assumed an investment cost of £10/MW for investment in all six generators. You can set the maximum amount of new capacity investment, which will be the same in all locations. The table below the line diagram gives the optimal investment levels (in MW) for each of the generators.

Experiment with this. Where does investment take place, depending on your chosen parameters? How does generation investment interact with line capacity? What happens to prices if you allow investment? Do you understand these changes?

In [4]:
from IPython.display import IFrame
IFrame('https://eee-apps-836a08e1d2dd.herokuapp.com/electricity-investment-1', width='100%', height=710)

Of course, this model is very simple. A more realistic model could include
- multiple time periods, as in the first example on this notebook
- some of the technical constraints on generators that we have explored in previous weeks.
- a demand function, instead of perfectly inelastic demand. This would make the model quadratic, as the objective would now be a maximisation of social welfare.
- a representation of uncertainty about fuel costs, demand, etc., e.g., by having a set of scenarios for each of these parameters, with the objective minimising expected costs. This would make it a stochastic model.
- multiple investment stages (e.g., with investment decisions happening every few years), which gives the model the option to not invest right now but wait and see what will happen to some of the uncertain parameters. This would make it a multi-stage model.
- a measure of risk aversion, such that the objective would not be to minimise expected costs but some risk metric. This would make it a stochastic riskaverse model or, if we care about expected costs but want the model to come up with a feasible solution in all cases, a robust model.
- more levels, e.g. an explicit representation of the system operator engaging in redispatch; line outages, etc,
- game theory elements, e.g. different investors in a market that is imperfectly competitive. This is harder, as we then need to solve a number of optimisation models simultaneously, making the problem an equilibrium problem.

The flexible nature of these type of models mean that almost anything can be included (but if you make models too large and complex, they will become difficult or impossible to solve, and interpreting the results will be difficult in any case!). There is specialist optimisation software (e.g., GAMS, AIMMS, Fico Xpress, etc.) that can solve these problems efficiently; in addition, more and more packages for open-source alternatives such Python and Julia are being released, which can help build and solve a model quickly.

## Renewable Energy Support Schemes

http://www.open-electricity-economics.org/book/text/08.html#renewable-energy-support-schemes

## Renewable Energy Auction Design

http://www.open-electricity-economics.org/book/text/09.html#renewable-energy-auction-design